In [4]:
import numpy as np
import pandas as pd
import ESMF
import os
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cpf
plt.style.use('default')
import IOverticalGrid
import seaborn as sns
import grd
import configM2R
import xarray as xr
import glob

This is a test script to enable interpolation directly from sigma coordinates to z coordinates.
The goal is to be able to use ROMS results files as forcing using model2roms.

Trond Kristiansen, 12.04.2021, 13.04.2021

In [5]:
def remove_all_small_variables(ds):
    """ remove all the variables that have less than 3 dimensions

        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    for v in ds.variables:
        if v not in ['Cs_r', 'Cs_w', 'hc', 'Vtransform', 'ocean_time']:
            ds = ds.drop_vars(v) if len(ds[v].dims) < 3 else ds
    return ds

def select_interior(ds):
    """
    discard "exterior" u,v,rho-points to build a symetric grid

        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    ds = ds.isel(xi_rho=slice(1,-1), eta_rho=slice(1,-1))
    if 'xi_v' in ds.dims:
        ds = ds.isel(xi_v=slice(1,-1))
    if 'eta_u' in ds.dims:
        ds = ds.isel(eta_u=slice(1,-1))
    return ds

def rename_dims(ds):
    """ rename dimensions

        Parameters:
            ds (xarray.Dataset): ROMS dataset

    """
    ds = ds.rename({'xi_rho': 'xh', 'xi_v': 'xh', 'xi_u': 'xq',
                    'eta_rho': 'yh', 'eta_v': 'yq', 'eta_u': 'yh',
                    'ocean_time': 'time'
                    })
    return ds

def compute_depth_layers(ds, grid, hmin=0.1):
    """ compute depths of ROMS vertical levels (Vtransform = 2) """

    # compute vertical transformation functional
    S_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    S_w = (ds.hc * ds.s_w + ds.Cs_w * ds.h) / (ds.hc + ds.h)

    # compute depth of rho (layers) and w (interfaces) points
    z_rho = ds.zeta + (ds.zeta + ds.h) * S_rho
    z_w = ds.zeta + (ds.zeta + ds.h) * S_w

    # transpose arrays and fill NaNs with a minimal depth
    ds['z_rho'] = z_rho.transpose(*('time', 's_rho','yh','xh'),
                                  transpose_coords=False).fillna(hmin)

    ds['z_w'] = z_w.transpose(*('time', 's_w','yh','xh'),
                                  transpose_coords=False).fillna(hmin)

    # interpolate depth of levels at U and V points
    ds['z_u'] = grid.interp(ds['z_rho'], 'X', boundary='fill')
    ds['z_v'] = grid.interp(ds['z_rho'], 'Y', boundary='fill')

    # compute layer thickness as difference between interfaces
    ds['dz'] = grid.diff(ds['z_w'], 'Z')

    # add z_rho and z_w to xarray coordinates
    ds = ds.set_coords(['z_rho', 'z_w', 'z_v', 'z_u'])

    return ds

def add_coords(ds):
    """ set coordinate variables as xarray coordinates

        Parameters:
            ds (xarray.Dataset): ROMS dataset
    """
    ds = ds.set_coords(['Cs_r', 'Cs_w', 'hc', 'h', 'Vtransform', 'time',
                        'lon_rho', 'lon_v', 'lon_u', 'lon_psi',
                        'lat_rho', 'lat_v', 'lat_u', 'lat_psi'])
    return ds

In [6]:
# https://raphaeldussin.medium.com/modern-python-tools-for-the-roms-ocean-model-bfca8642db01
# use glob to create a list of files to convert
filelist = glob.glob('Examples/Grids/norkyst_800m*.nc*')
# path to grid file
gridfile = 'Examples/Grids/norkyst_800m_his.nc4_2019121501-2019121600'
# output store name
store_name = 'CCS2_store'

# load the grid
ds_grid = xr.open_dataset(gridfile)
# drop some unnecessary variables
ds_grid = ds_grid.drop_vars(['Cs_r', 'Cs_w', 'hc'])
# select interior points for grid variables
ds_grid = select_interior(ds_grid)

first=True
# iterate on files
for ncfile in filelist:
    print("Iteration for file ",ncfile)
    ds = xr.open_dataset(ncfile)
    print("Ite 1")
    ds = remove_all_small_variables(ds)
    print("Ite 2")
    ds = select_interior(ds)
    print("Ite 3")
    ds = xr.merge([ds, ds_grid])
    print("Ite 4")
    ds = rename_dims(ds)
    print("Ite 5")
    ds = add_coords(ds)
    print("Ite 6")
    ds = ds.chunk({'time': 1})
    print("Ite 7")
    print(f"working on {ncfile}")
    if first:
        ds.to_zarr(store_name, consolidated=True, mode='w')
    else:
        ds.to_zarr(store_name, consolidated=True, mode='a', append_dim='time')
    first=False

Iteration for file  Examples/Grids/norkyst_800m_his.nc4_2019121501-2019121600
Ite 1
Ite 2
Ite 3
Ite 4
Ite 5


ValueError: One or more of the specified variables cannot be found in this dataset

In [ ]:
import xarray as xr
from xgcm import Grid

CCS2 = xr.open_zarr(CCS2_store, consolidated=True)

# Create xgcm grid object
grid_ccs2 = Grid(CCS2, coords={'X': {'center': 'xh', 'outer': 'xq'},
                               'Y': {'center': 'yh', 'outer': 'yq'},
                               'Z': {'center': 's_rho', 'outer': 's_w'}},
                 periodic=False)

# Add depths of layers and interfaces to dataset
CCS2 = compute_depth_layers(CCS2, grid_ccs2)

In [ ]:
to_grid_file="Examples/Grids/norfjords_160m_grid.nc_A04"
from_grid_file="Examples/Grids/norkyst_800m_his.nc4_2019121501-2019121600"

ds=xr.open_dataset(from_grid_file)
confM2R = configM2R.Model2romsConfig()

print(confM2R.hc,confM2R.vtransform)
grd_obj = grd.Grd("ROMS",confM2R=confM2R)

grd_obj.create_object(confM2R, from_grid_file)

IOverticalGrid.calculateVgrid(grd_obj)

ax=plt.figure(figsize=(8,8)).gca(projection=ccrs.PlateCarree())
print(grd_obj.lon_rho)
print(grd_obj.lat_rho)
print(np.shape(grd_obj.z_r))
cs= ax.pcolormesh(grd_obj.lon_rho.values,
                  grd_obj.lat_rho.values,
                  grd_obj.z_r[0],
                  cmap=sns.color_palette("Spectral_r", as_cmap=True),
                          transform=ccrs.PlateCarree())
plt.colorbar(cs, shrink=.4)
ax.coastlines()
#lons,lats =np.meshgrid(grd_obj.lon_rho,grd_obj.lat_rho)
#step=12
#ax.scatter(lons[::step,::step],lats[::step,::step],3,c="w")
#plt.title("test")
plt.show()

In [ ]:
lons = grd_obj.lon_rho.values.ravel()
lats = grd_obj.lat_rho.values.ravel()
print(len(lats),np.shape(grd_obj.z_r)[0])
z = grd_obj.z_r.ravel().reshape(len(lats),np.shape(grd_obj.z_r)[0])

print(np.shape(lons),np.shape(lats),np.shape(z))
grid_create_from_coordinates_3d(lons,lats,z)

#for lon, lat, in zip(ds.lon_rho, ds.lat_rho):

